<a href="https://colab.research.google.com/github/vitaliy-sharandin/data_science_projects/blob/master/portfolio/eda/AI_risks_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI risks EDA
This project provides analysis of AI related incidents throughout the years.

# Datasets
The dataset used in project is based on the website aggregating AI incidents:
https://incidentdatabase.ai/


* AI Incident Database <br>
https://www.kaggle.com/datasets/konradb/ai-incident-database


In [24]:
!pip install -U -q datasets
!pip install -U -q ydata-profiling
!pip install -U -q keybert
!pip install -U -q keyphrase-vectorizers
!pip install -U -q spacy
!python -m spacy download en_core_web_lg

2023-10-13 10:20:19.990629: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-13 10:20:22.321055: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [25]:
from datasets import load_dataset
import typing_extensions
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import Counter
from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseTfidfVectorizer
from sentence_transformers import SentenceTransformer, util
import spacy
import typing_extensions

ImportError: ignored

In [16]:
pip freeze typing_extensions

absl-py==1.4.0
aiohttp==3.8.6
aiosignal==1.3.1
alabaster==0.7.13
albumentations==1.3.1
altair==4.2.2
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array-record==0.4.1
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
attrs==23.1.0
audioread==3.0.1
autograd==1.6.2
Babel==2.13.0
backcall==0.2.0
beautifulsoup4==4.11.2
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.2.2
bqplot==0.12.40
branca==0.6.0
build==1.0.3
CacheControl==0.13.1
cachetools==5.3.1
catalogue==2.0.10
certifi==2023.7.22
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.0
chex==0.1.7
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.15.1
cloudpickle==2.2.1
cmake==3.27.6
cmdstanpy==1.2.0
colorcet==3.0.1
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.3
cons==0.4.6
contextlib2==21.6.0
contourpy==1.1.1
convertdate==2.4.0
cryptography==41.0.4
cufflinks==0.17.3
cupy-cuda11x==11.0.0
cvxopt==1.3.2
cvxpy==1.3.2
cycler==0.12.1
cymem==2.

In [ ]:
incident_dataset = load_dataset("vitaliy-sharandin/ai-incidents")
incident_dataset = incident_dataset['train'].to_pandas()

In [ ]:
profile = ProfileReport(incident_dataset, title="Fraud data report", dark_mode=True)
profile.to_notebook_iframe()

# Distribution of incidents over time

## Amount of incidents through time

In [ ]:
incident_dataset['date'] = pd.to_datetime(incident_dataset['date'])
incident_dataset['year'] = incident_dataset['date'].dt.year
plt.figure(figsize=(12, 6))
sns.histplot(incident_dataset['year'], bins=40, kde=True)
plt.title('Distribution of AI Incidents Over Time')
plt.xlabel('Year')
plt.ylabel('Number of Incidents')
plt.grid(True)
plt.show()

**Insights**
* Early Years <br>
There are very few incidents reported before the year 2010 which can be related to the fact that deep learning models boom happened in early 2010s.

* Rapid Increase <br>
There is a noticeable increase in the number of incidents starting from around 2015. This correlates with wider adoption of models as well as development of advanced programs such as AlphaGo and introduction of transformer arhitecture in 2017.

In [ ]:
earliest_incident_year = incident_dataset['year'].min()
incident_dataset[incident_dataset['year']==earliest_incident_year]

Earliest incident happended in 1983 when Stanislav Petrov identified incoming intercontinental ballistic missiles shown by the automatic aerial strike warning system flying at Soviet Union as false positive, preventing nuclear armageddon.

# Deployers and developers of models

In [ ]:
def count_occurrences(column):
    items = column.str.replace('[\[\]"]', '', regex=True).str.split(',')
    counter = Counter([item.strip() for sublist in items.dropna() for item in sublist])
    return counter

deployer_counts = count_occurrences(incident_dataset['Alleged deployer of AI system'])
developer_counts = count_occurrences(incident_dataset['Alleged developer of AI system'])


deployer_df = pd.DataFrame(deployer_counts.items(), columns=['Deployer', 'Count']).sort_values('Count', ascending=False).head(30)
developer_df = pd.DataFrame(developer_counts.items(), columns=['Developer', 'Count']).sort_values('Count', ascending=False).head(30)


fig, axes = plt.subplots(1, 2, figsize=(20, 7))

sns.barplot(x='Count', y='Deployer', data=deployer_df, ax=axes[0])
axes[0].set_title('Top 10 Deployers Involved in AI Incidents')
axes[0].set_xlabel('Number of Incidents')

sns.barplot(x='Count', y='Developer', data=developer_df, ax=axes[1])
axes[1].set_title('Top 10 Developers Involved in AI Incidents')
axes[1].set_xlabel('Number of Incidents')

plt.tight_layout()
plt.show()

**Insights**

  * "Facebook" appears to be both the most frequent deployer involved in AI incidents, followed by "Tesla" and "Google", as platforms with large user bases and extensive AI deployments are more prone to incidents, especially Facebook which knowingly feeds their machine learning models with user data.

  * "Facebook", "Tesla", "OpenAI" and "Google" are among the top developers involved in AI incidents. This isn't surprising given their role as major technology companies with extensive AI research and deployment.
  
  * Noticeably, many of the top deployers and developers are tech giants, which could imply a higher level of responsibility for these organizations in ensuring AI safety and ethics.

# Harmed parties

In [ ]:
harmed_counts = count_occurrences(incident_dataset['Alleged harmed or nearly harmed parties'])

harmed_df = pd.DataFrame(harmed_counts.items(), columns=['Harmed Party', 'Count']).sort_values('Count', ascending=False).head(20)

plt.figure(figsize=(20, 7))
sns.barplot(x='Count', y='Harmed Party', data=harmed_df)
plt.title('Top 10 Most Commonly Harmed Parties in AI Incidents')
plt.xlabel('Number of Incidents')

plt.show()

**Insights**


  * The category "facebook-users" tops the list which correlates with what we saw previously in deployers and developers analysis, meaning Facebook is harming its own users. Also, we can see that other big platform users are often harmed by their owners.

  * We can see that minority groups in general, racial groups such as Jewish and Black as well as women are affected by biases of AI.

  * Categories like "pedestrians", "tesla-drivers" and "motorists" are contributing to incidents in automotive industry.

# AI incident types

## Incident keywords extraction

Below you can see the keywords extraction from either title or description of incident indicating what was the most probable reason of incident in one phrase or word.<br>
Here `KeyBert` together with `KeyphraseTfidfVectorizer` were used for finding max similarity to the words in the filtering list containning different generalized incident phrases. Manual verification of result was used for calibration of this method which in the end performs pretty well.

In [ ]:
import traceback

nlp = spacy.load('en_core_web_lg')

kw_model = KeyBERT()
kph_vectorizer = KeyphraseTfidfVectorizer(spacy_pipeline=nlp, pos_pattern = '<J.*>*<V.*>*<N.*>*')

st_model = SentenceTransformer('distilbert-base-nli-mean-tokens')

# keyword filter
failures_filter = ["wrong action","negative cause","failure reason","AI failure","software failure",
                  "bad", "fail", "crash", "stop", "halt", "break", "malfunction", "kill", "die",
                  "hurt","bias","disrespect","discrimination","racism","sexism","wrong","error"]

def keybert_failures_combined(row, similarity_threshold=0.6, filter_phrases=failures_filter):

  def get_most_relevant(text):
      keywords_tuples = kw_model.extract_keywords(text, vectorizer=kph_vectorizer)
      keywords = [kw[0] for kw in keywords_tuples if kw]

      keyword_embeddings = st_model.encode(keywords, convert_to_tensor=True)
      filter_phrase_embeddings = st_model.encode(filter_phrases, convert_to_tensor=True)

      max_similarities = [max([util.pytorch_cos_sim(keyword_embedding, filter_phrase_embedding).item()
                                for filter_phrase_embedding in filter_phrase_embeddings])
                          for keyword_embedding in keyword_embeddings]

      if not max_similarities:
          return None, 0

      most_relevant_keyword = keywords[max_similarities.index(max(max_similarities))]
      return most_relevant_keyword, max(max_similarities)

  desc_keyword, desc_similarity = get_most_relevant(row['description'])
  title_keyword, title_similarity = get_most_relevant(row['title'])

  return desc_keyword if desc_similarity >= title_similarity else title_keyword

incident_dataset['ai_failures_summary'] = incident_dataset.apply(keybert_failures_combined, axis=1)

incident_dataset[['description','title','ai_failures_summary']]

## Incidents cloud

Now we create a cloud of words to see the most frequent out of those incident reasons we have found in previous step as a quick and simple glimpse onto main types of incidents.

In [ ]:
from wordcloud import WordCloud

wordcloud = WordCloud(
    background_color='black',
    max_words=100,
    width=800,
    height=400
).generate(' '.join(incident_dataset['ai_failures_summary']))

# Plotting the word cloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud for Incident Keywords')
plt.show()

## Cluster analysis
Now let's form groups out of incident reasons with cluster analysis.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

def get_single_term_for_cluster(data, clusters, labels):
    """
    Returns a dictionary of cluster centers (mean TF-IDF vector of each cluster) and their top keyword.
    It aims to capture the most distinguishing feature of each cluster.
    """
    df = pd.DataFrame(data.todense()).groupby(clusters).mean()
    overall_mean = df.mean(axis=0)

    top_keywords = {}

    for i, r in df.iterrows():
        relative_difference = r - overall_mean
        top_keywords[i] = labels[np.argmax(relative_difference)]

    return top_keywords

def tokenize_and_filter(sentence):
  tokens = nlp(sentence)
  return [token.text for token in tokens if token.pos_ in ["NOUN", "ADJ", "ADV", "VERB"]]

def cluster_and_visualize(df, column="description", n_clusters=50):
    # Vectorization
    tfidf_vectorizer = TfidfVectorizer(max_df=0.85, max_features=10000, tokenizer=tokenize_and_filter)
    tfidf_matrix = tfidf_vectorizer.fit_transform(df[column])

    # KMeans Clustering
    km = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = km.fit_predict(tfidf_matrix)

    display(clusters)

    # Determine Cluster Names
    cluster_keywords = get_single_term_for_cluster(tfidf_matrix, clusters, tfidf_vectorizer.get_feature_names_out())
    df['Cluster'] = [cluster_keywords[cluster] for cluster in clusters]

    display(df['Cluster'])

    # Visualization - PCA plot
    pca = PCA(n_components=2)
    reduced_features = pca.fit_transform(tfidf_matrix.toarray())

    plt.figure(figsize=(15, 7))
    scatter = plt.scatter(reduced_features[:, 0], reduced_features[:, 1], c=clusters, cmap='viridis', s=50)
    plt.title('PCA Visualization of AI Failure Descriptions Clusters')
    plt.xlabel('PCA 1')
    plt.ylabel('PCA 2')
    legend_labels = [cluster_keywords[i] for i in range(n_clusters)]
    plt.legend(handles=scatter.legend_elements()[0], labels=legend_labels, title="Clusters")
    plt.show()

    # Visualization - Cluster Count plot
    top_clusters = df['Cluster'].value_counts().head(10).reset_index()
    top_clusters.columns = ['Cluster', 'Count']

    plt.figure(figsize=(15, 7))
    sns_barplot = sns.barplot(x='Count', y='Cluster', data=top_clusters)
    plt.title('Counts of Top 10 AI Failure Descriptions Clusters')
    plt.xlabel("Count")
    plt.ylabel("Cluster")

    # Display counts on the bars
    for index, value in enumerate(top_clusters['Count']):
        sns_barplot.text(value, index, f' {value}', color='black', ha="left", va="center")

    plt.show()

    return df

incident_dataset = cluster_and_visualize(incident_dataset, column='ai_failures_summary')

**Insights**

We can see that main AI failures are caused by:

* Inaccuracy
* Unrealiability
* Bias
* Falsehood
* Crashes
* Company related activity
* Racism



# Clusters through time



In [ ]:
def visualize_clusters_stacked_bar_yearly(df, date_column="date"):
    # Extract year from 'date' column
    df['Year'] = pd.to_datetime(df[date_column]).dt.year

    # Extract top 10 clusters
    top_clusters = df['Cluster'].value_counts().head(10).index.tolist()

    # Filter dataframe to only include top 10 clusters
    top_clusters_df = df[df['Cluster'].isin(top_clusters)]

    # Pivot the data to get years as rows and clusters as columns
    pivot_table = top_clusters_df.groupby(['Year', 'Cluster']).size().unstack().fillna(0)

    # Sort columns based on their total count across all years
    sorted_columns = pivot_table.sum().sort_values(ascending=False).index
    pivot_table = pivot_table[sorted_columns]

    # Plotting
    plt.figure(figsize=(15, 8))
    bottom = np.zeros(len(pivot_table))

    for cluster in sorted_columns:
        plt.bar(pivot_table.index, pivot_table[cluster], bottom=bottom, label=cluster)
        bottom += pivot_table[cluster].values

    plt.title('Yearly Counts of Top 10 AI Failure Descriptions Clusters')
    plt.ylabel('Count')
    plt.xlabel('Year')
    plt.xticks(pivot_table.index, rotation=45)  # Ensure each year is displayed
    plt.gca().yaxis.grid(False)    # Remove the y-axis grid
    plt.gca().xaxis.grid(False)    # Remove the x-axis grid
    plt.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

visualize_clusters_stacked_bar_yearly(incident_dataset)

In [ ]:
# from datasets import Dataset

# # Convert the pandas dataframe to a Hugging Face dataset
# hf_dataset = Dataset.from_pandas(incident_dataset)

# # Push the dataset to the Hugging Face Hub
# hf_dataset.push_to_hub(
#     repo_id="vitaliy-sharandin/ai-incidents"
# )